In [1]:
import psycopg2
# Connect to an existing database
conn = psycopg2.connect("dbname='bt' user='postgres' host='localhost' password='severo' port='5432'")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a command: this creates a new table
cur.execute("select m.tid,m.time,m.edge_id from matched_tracks m order by m.tid,m.time;")
matched_tracks = cur.fetchall()
cur.execute("select distinct edge_id from matched_tracks group by edge_id order by edge_id asc;")
egde_ids = cur.fetchall()
cur.execute("select i.tid,i.start_time,i.end_time,EXTRACT(EPOCH FROM (i.end_time::timestamp - i.start_time::timestamp)) from ib_stops_carlos i order by i.tid, start_time;")
stops = cur.fetchall()
# Pass data to fill a query placeholders and let Psycopg perform
# the correct conversion (no more SQL injections!)
#>>> cur.execute("INSERT INTO test (num, data) VALUES (%s, %s)",
#...      (100, "abc'def"))

# Query the database and obtain data as Python objects
#>>> cur.execute("SELECT * FROM test;")
#>>> cur.fetchone()
#(1, 100, "abc'def")

# Make the changes to the database persistent
#>>> conn.commit()

# Close communication with the database
cur.close()

conn.close()

In [2]:
# função que verificar se um ponto é um stop.
def verifica(ponto,stops):
    for stop in stops:
        if ponto[0]==stop[0] and ponto[1]==stop[1]:
            return stop[3]
    return -1  

In [3]:
# Encrontando as trajetorias.
trajetorias = list()
trajetoria = list()
taxi_id = 1

for ponto in matched_tracks:
    if ponto[0]==taxi_id:
        diferenca=verifica([ponto[0],ponto[1]],stops)
        if diferenca==-1:
            trajetoria.append(ponto)
        else:
            trajetoria.append(ponto)
            if diferenca > 120:
                trajetorias.append(trajetoria)
                trajetoria = list()
    else:
        trajetorias.append(trajetoria)
        trajetoria = list()
        trajetoria.append(ponto)
        taxi_id=taxi_id+1     

In [4]:
# Criando dicionário para mapeamento dos egde_ids para os indices da matriz m:
dicionario1=dict()
indice=0
for egde_id in egde_ids:
    dicionario1[egde_id[0]]=indice
    indice=indice+1
# Criando dicionário para o mapeamento dos indices da matriz para os egdes_ids:
dicionario2 = dict()
for i in egde_ids:
    d=dicionario1[i[0]]
    dicionario2[d]=i[0]

In [5]:
# Criando a matriz de transições:
import numpy as np
k=len(dicionario1)
m =np.zeros( (k,k),dtype=np.int32)

In [6]:
# Contando quantas transições exites de cada egde_id para outro:
for traj in trajetorias:
    c = len(traj)
    for i in range(0,c-1):
        if traj[i][2] != traj[i+1][2]:
            d1=dicionario1[traj[i][2]]
            d2=dicionario1[traj[i+1][2]]
            m[d1][d2]=m[d1][d2]+1

In [7]:
# Contando a quantidade de celulas não nulas.
count=0
for n in m:
    for i in n:
        if i!= 0:
            count=count+1

In [8]:
# Calculando as dimensões da matriz para mapeamento.
import math
raiz=math.ceil(math.sqrt(count))
# Criando a matriz para o mapeamento.
n =np.zeros( (raiz,raiz),dtype=np.int32)

In [9]:
# Mapeamento de uma matriz para outra.
dicionario3=dict()
l=0
c=0
for i in range(0,k):
    for j in range(0,k):
        if m[i][j] !=0:
            n[l][c]=m[i][j]
            dicionario3[(l,c)]=(i,j)
            c=c+1
            if c==raiz:
                c=0
                l=l+1

In [15]:
def not_marked(m,a):
    for n in m:
        for i in n:
            if i > 0 and i>=a:
                return True
    return False

def maximo(m,c):
    l=0
    f=0
    for i in range(0,c):
        for j in range(0,c):
            if m[i][j]>m[l][f]:
                l=i
                f=j
    return [l,f]

def marking(a):
    if a>0:
        return -1*a
    return a

def not_marked_2(m,d,f,a,e,c):
    u=-1
    minimo=100000
    for i in range(0,c):
        if m[f][i]>=a:
            t = m[d][f]
            if t<0:
                t=t*(-1)
            if abs(t-m[f][i]) < minimo and abs(t-m[f][i])<=e:
                minimo = abs(t-m[f][i])
                u=i
    return u

def not_marked_3(m,d,f,a,e,c):
    u=-1
    minimo=1000000
    for i in range(0,c):
        if m[i][d]>=a:
            t = m[d][f]
            if t<0:
                t=t*(-1)
            if abs(t-m[u][d])<minimo and abs(t-m[u][d])<=e:
                minimo=abs(t-m[u][d])
                u=i
    return u

In [14]:
def netscan_phase_1(dicionario3,c,dicionario2,m,e,a):
    O=set()
    k=0
    C=list()
    while not_marked(m,a):
        indices=maximo(m,c)
        d=indices[0]
        f=indices[1]
        tupla=dicionario3[(d,f)]
        Sd=dicionario2[tupla[0]]
        print("Sd: ",Sd)
        Sf=dicionario2[tupla[1]]
        print("Sf: ",Sf)
        C.append([Sd,Sf])
        m[d][f]=marking(m[d][f])
        u=not_marked_2(m,d,f,a,e,c)
        while u>-1:
            print("1")
            tupla=dicionario3[(f,u)]
            Su=dicionario2[tupla[1]]
            print("Su :",Su)
            C[k].append(Su)
            m[f][u]=marking(m[f][u])
            d=f
            f=u
            u=not_marked_2(m,d,f,a,e,c)
        u=not_marked_3(m,d,f,a,e,c)
        while u>-1:
            print("2")
            tupla=dicionario3[(u,d)]
            Su=dicionario2[tupla[0]]
            print("Su :",Su)
            C[k] = [Su]+C[k]
            m[u][d]=marking(m[u][d])
            f=d
            d=u
            u=not_marked_3(m,d,f,a,e,c)
        k=k+1
    return C

In [17]:
C=netscan_phase_1(dicionario3,raiz,dicionario2,n,10,3)

Sd:  23871
Sf:  23872
Sd:  23872
Sf:  23861
Sd:  46892
Sf:  23869
Sd:  23869
Sf:  23870
Sd:  23870
Sf:  23871
Sd:  44946
Sf:  23862
Sd:  23867
Sf:  23868
1
Su : 40132
1
Su : 19180
1
Su : 153
1
Su : 16160
1
Su : 47290
1
Su : 6327
1
Su : 17
1
Su : 33487
1
Su : 3604
1
Su : 24349
1
Su : 270
1
Su : 23687
2
Su : 826
2
Su : 154
2
Su : 1777
2
Su : 830
2
Su : 2196
2
Su : 2876
2
Su : 18
2
Su : 1776
2
Su : 2875
2
Su : 832
2
Su : 6270
2
Su : 840
2
Su : 828
2
Su : 6561
2
Su : 2210
2
Su : 17
2
Su : 3533
2
Su : 834
2
Su : 6752
2
Su : 292
2
Su : 152
Sd:  23868
Sf:  44946
Sd:  29745
Sf:  29746
1
Su : 40029
1
Su : 43197
1
Su : 18933
1
Su : 8609
1
Su : 47693
1
Su : 24709
1
Su : 40359
1
Su : 3565
1
Su : 16159
1
Su : 46888
1
Su : 27282
1
Su : 6658
1
Su : 20706
1
Su : 2872
1
Su : 5166
1
Su : 35041
1
Su : 53124
1
Su : 558
1
Su : 27549
1
Su : 1204
1
Su : 83225
1
Su : 19178
1
Su : 36281
1
Su : 1207
1
Su : 24668
1
Su : 31438
1
Su : 5167
1
Su : 6128
1
Su : 37809
1
Su : 7725
1
Su : 47159
1
Su : 41317
1
Su : 173
1

Su : 30360
1
Su : 6224
1
Su : 39862
1
Su : 32540
1
Su : 29840
1
Su : 37988
1
Su : 24548
1
Su : 3827
1
Su : 28828
1
Su : 3357
1
Su : 28411
1
Su : 37246
1
Su : 24065
1
Su : 25355
1
Su : 6561
1
Su : 711
1
Su : 41345
1
Su : 38110
1
Su : 9494
1
Su : 43207
1
Su : 27503
1
Su : 19109
1
Su : 26203
1
Su : 25378
1
Su : 23951
1
Su : 51414
1
Su : 2173
1
Su : 51607
1
Su : 7727
1
Su : 5071
1
Su : 41318
1
Su : 5124
1
Su : 52992
1
Su : 19284
1
Su : 6664
1
Su : 29999
1
Su : 24344
1
Su : 53377
1
Su : 31573
1
Su : 28352
1
Su : 25642
1
Su : 21499
1
Su : 12989
1
Su : 31268
1
Su : 43208
1
Su : 23772
2
Su : 827
2
Su : 23863
Sd:  37553
Sf:  23866
1
Su : 25383
1
Su : 29420
1
Su : 44671
1
Su : 53137
1
Su : 6661
1
Su : 31922
1
Su : 24516
1
Su : 34630
1
Su : 2172
1
Su : 51176
1
Su : 30070
1
Su : 9493
1
Su : 38220
1
Su : 36753
1
Su : 7739
1
Su : 23737
1
Su : 4941
1
Su : 12024
1
Su : 41315
1
Su : 51162
1
Su : 3853
1
Su : 38608
1
Su : 55323
1
Su : 24550
1
Su : 51412
1
Su : 1210
1
Su : 46915
1
Su : 52793
1
Su : 41171


Sd:  2239
Sf:  2240
1
Su : 25382
1
Su : 22850
1
Su : 28608
1
Su : 9772
1
Su : 39542
1
Su : 36799
1
Su : 34123
1
Su : 40363
1
Su : 6267
1
Su : 109
1
Su : 35063
1
Su : 7409
1
Su : 46475
1
Su : 37885
1
Su : 10424
1
Su : 18935
1
Su : 10377
2
Su : 2219
2
Su : 38304
2
Su : 3211
2
Su : 28329
2
Su : 2251
Sd:  8606
Sf:  8607
1
Su : 46891
1
Su : 47289
1
Su : 3487
1
Su : 16185
2
Su : 20758
2
Su : 4969
2
Su : 35059
2
Su : 207
2
Su : 26321
2
Su : 3607
2
Su : 21332
2
Su : 10425
2
Su : 24662
2
Su : 7239
2
Su : 3318
2
Su : 21549
2
Su : 3599
2
Su : 28330
2
Su : 3827
2
Su : 24629
2
Su : 18679
2
Su : 508
2
Su : 6559
2
Su : 24633
2
Su : 2929
2
Su : 24625
2
Su : 18946
2
Su : 1224
2
Su : 24349
2
Su : 3820
2
Su : 21333
2
Su : 18938
2
Su : 4966
2
Su : 35293
2
Su : 1271
2
Su : 32763
2
Su : 3198
2
Su : 30077
2
Su : 2255
2
Su : 40355
2
Su : 226
2
Su : 34264
2
Su : 3201
2
Su : 34269
2
Su : 3404
2
Su : 16187
2
Su : 24665
2
Su : 21388
2
Su : 3598
2
Su : 28828
2
Su : 3375
2
Su : 26329
2
Su : 6653
2
Su : 2886
2
Su : 

Su : 18673
2
Su : 2920
Sd:  46610
Sf:  46611
Sd:  46611
Sf:  46612
1
Su : 3321
1
Su : 5618
1
Su : 13580
1
Su : 43439
1
Su : 34349
1
Su : 33288
1
Su : 6679
1
Su : 37402
1
Su : 28357
1
Su : 10372
1
Su : 25381
1
Su : 21319
1
Su : 79291
1
Su : 249
1
Su : 51049
1
Su : 1187
1
Su : 52695
1
Su : 34461
1
Su : 54229
1
Su : 422
1
Su : 36578
1
Su : 57124
1
Su : 27577
1
Su : 8194
1
Su : 34628
1
Su : 9781
1
Su : 709
1
Su : 6991
1
Su : 27022
1
Su : 32301
1
Su : 47103
1
Su : 8568
1
Su : 40728
1
Su : 12666
1
Su : 56431
1
Su : 427
1
Su : 41214
1
Su : 6666
1
Su : 36644
1
Su : 23976
1
Su : 38209
1
Su : 26169
1
Su : 78661
1
Su : 50080
1
Su : 27735
1
Su : 24073
1
Su : 33418
1
Su : 29327
2
Su : 202
2
Su : 42649
2
Su : 7669
2
Su : 3319
2
Su : 39850
2
Su : 3087
2
Su : 35052
2
Su : 3406
2
Su : 40114
Sd:  46613
Sf:  57144
1
Su : 24720
1
Su : 28095
1
Su : 6676
1
Su : 2175
1
Su : 60260
1
Su : 23858
1
Su : 75989
1
Su : 49747
1
Su : 19070
1
Su : 42884
2
Su : 3216
2
Su : 41486
2
Su : 125
Sd:  3606
Sf:  3607
1
Su : 25

Su : 6225
2
Su : 839
2
Su : 35048
2
Su : 24712
2
Su : 41239
2
Su : 3219
2
Su : 51589
2
Su : 3116
2
Su : 37636
2
Su : 5066
2
Su : 55165
2
Su : 420
2
Su : 43543
2
Su : 13042
2
Su : 24923
2
Su : 24686
2
Su : 39883
2
Su : 3214
2
Su : 58174
2
Su : 423
2
Su : 50316
2
Su : 5119
2
Su : 51592
2
Su : 3504
2
Su : 50066
2
Su : 2131
2
Su : 35572
2
Su : 375
2
Su : 79293
2
Su : 3132
2
Su : 38305
2
Su : 21436
2
Su : 32212
2
Su : 1267
2
Su : 47278
2
Su : 2129
2
Su : 43209
2
Su : 7670
2
Su : 18929
Sd:  34904
Sf:  34905
Sd:  34905
Sf:  46874
1
Su : 20758
1
Su : 33189
1
Su : 51524
1
Su : 36108
1
Su : 34267
1
Su : 838
1
Su : 5170
1
Su : 7665
1
Su : 40119
1
Su : 5172
1
Su : 29627
1
Su : 60261
1
Su : 37244
1
Su : 51174
1
Su : 304
1
Su : 3202
1
Su : 32353
1
Su : 57637
1
Su : 37457
1
Su : 35304
1
Su : 5612
1
Su : 5213
1
Su : 14488
1
Su : 35160
1
Su : 7705
1
Su : 34320
1
Su : 37295
1
Su : 36231
1
Su : 13582
1
Su : 33495
1
Su : 45193
1
Su : 33308
1
Su : 37765
1
Su : 6248
1
Su : 1188
1
Su : 45297
1
Su : 43139
1
S

...

Sd:  43432
Sf:  7022
Sd:  43837
Sf:  43838
2
Su : 36105
2
Su : 376
Sd:  47113
Sf:  24729
1
Su : 16170
2
Su : 39730
2
Su : 36094
Sd:  51553
Sf:  51554
1
Su : 45309
1
Su : 58480
1
Su : 73033
2
Su : 40034
2
Su : 27504
Sd:  51554
Sf:  51555
1
Su : 29775
1
Su : 47160
1
Su : 32374
1
Su : 29107
1
Su : 43227
2
Su : 32594
2
Su : 46969
2
Su : 34485
2
Su : 2299
Sd:  51556
Sf:  51557
1
Su : 46724
1
Su : 35296
2
Su : 49802
2
Su : 6994
Sd:  55109
Sf:  55110
1
Su : 24152
1
Su : 7239
1
Su : 38294
1
Su : 46604
2
Su : 2127
2
Su : 39474
2
Su : 47155
2
Su : 34809
Sd:  57566
Sf:  57567
1
Su : 24172
2
Su : 54569
2
Su : 27842
Sd:  57567
Sf:  57568
2
Su : 29093
2
Su : 38063
2
Su : 50674
2
Su : 26935
2
Su : 37678
Sd:  57696
Sf:  46972
1
Su : 40690
1
Su : 55168
1
Su : 5280
2
Su : 26045
2
Su : 37676
Sd:  297
Sf:  298
Sd:  2890
Sf:  28396
2
Su : 35289
2
Su : 51522
2
Su : 29898
2
Su : 47136
2
Su : 34486
Sd:  3127
Sf:  3203
Sd:  3133
Sf:  3134
Sd:  3324
Sf:  3325
Sd:  3810
Sf:  29817
Sd:  6661
Sf:  6662
Sd:  24347


Sd:  34444
Sf:  34445
1
Su : 27823
1
Su : 42868
1
Su : 56598
1
Su : 51182
2
Su : 46664
2
Su : 2389
Sd:  34606
Sf:  34607
Sd:  34607
Sf:  34608
Sd:  34829
Sf:  34830
1
Su : 52313
1
Su : 50668
1
Su : 36371
1
Su : 41109
1
Su : 49769
1
Su : 27421
1
Su : 40730
2
Su : 26497
2
Su : 18951
Sd:  34830
Sf:  34831
1
Su : 27501
1
Su : 53882
2
Su : 26517
Sd:  34831
Sf:  34832
1
Su : 39541
1
Su : 55181
2
Su : 34531
Sd:  34832
Sf:  34833
Sd:  34833
Sf:  34834
1
Su : 55163
2
Su : 31967
Sd:  34910
Sf:  47169
Sd:  35275
Sf:  35262
2
Su : 373
Sd:  35304
Sf:  35305
Sd:  35305
Sf:  35306
Sd:  35307
Sf:  35308
Sd:  37395
Sf:  37396
Sd:  37473
Sf:  37486
1
Su : 45840
2
Su : 34495
Sd:  37650
Sf:  37651
Sd:  37697
Sf:  37708
1
Su : 33532
1
Su : 75663
2
Su : 27853
Sd:  37763
Sf:  37764
Sd:  37765
Sf:  37766
1
Su : 56674
2
Su : 23994
Sd:  37768
Sf:  43509
1
Su : 58210
2
Su : 26065
2
Su : 43464
Sd:  37783
Sf:  37762
1
Su : 65882
2
Su : 28314
2
Su : 57948
2
Su : 31319
2
Su : 34440
Sd:  37787
Sf:  24701
1
Su : 1182


Sd:  46972
Sf:  57698
Sd:  47160
Sf:  47159
1
Su : 43228
2
Su : 37458
Sd:  47162
Sf:  47163
1
Su : 52928
2
Su : 43175
2
Su : 43172
2
Su : 51549
2
Su : 41282
2
Su : 37454
Sd:  47163
Sf:  47164
1
Su : 45649
2
Su : 38266
Sd:  51141
Sf:  51142
2
Su : 45650
2
Su : 38126
Sd:  51142
Sf:  28111
2
Su : 51372
2
Su : 51562
2
Su : 41437
2
Su : 32296
Sd:  51394
Sf:  24335
2
Su : 52779
2
Su : 37761
Sd:  51558
Sf:  41281
2
Su : 45295
2
Su : 34985
Sd:  51570
Sf:  51571
1
Su : 57639
1
Su : 42808
2
Su : 24739
Sd:  51571
Sf:  51572
1
Su : 23738
2
Su : 38287
Sd:  51572
Sf:  51573
2
Su : 46580
2
Su : 34483
Sd:  51574
Sf:  51575
2
Su : 47014
Sd:  51575
Sf:  51576
2
Su : 51373
2
Su : 21490
Sd:  51576
Sf:  51577
2
Su : 51563
2
Su : 47171
2
Su : 47009
Sd:  51577
Sf:  51578
2
Su : 6998
Sd:  51578
Sf:  51579
Sd:  51579
Sf:  51580
Sd:  51580
Sf:  51581
Sd:  51581
Sf:  47572
Sd:  52780
Sf:  51570
1
Su : 51613
2
Su : 39824
Sd:  52806
Sf:  28603
2
Su : 40312
Sd:  55608
Sf:  55609
1
Su : 47097
1
Su : 40093
2
Su : 360

Sd:  36153
Sf:  36154
Sd:  36154
Sf:  43013
Sd:  36398
Sf:  36399
Sd:  36399
Sf:  31907
Sd:  36513
Sf:  36521
Sd:  36559
Sf:  36560
Sd:  36578
Sf:  36579
Sd:  36579
Sf:  36580
Sd:  36732
Sf:  36733
Sd:  37451
Sf:  37452
Sd:  37452
Sf:  37453
Sd:  37453
Sf:  37454
Sd:  37483
Sf:  12025
Sd:  37546
Sf:  37547
Sd:  37644
Sf:  37645
Sd:  37645
Sf:  37936
Sd:  37648
Sf:  41356
Sd:  37739
Sf:  37740
1
Su : 45880
2
Su : 55617
Sd:  37743
Sf:  37744
Sd:  37744
Sf:  37745
Sd:  37745
Sf:  43438
Sd:  37769
Sf:  43426
Sd:  37779
Sf:  43432
Sd:  37968
Sf:  51254
1
Su : 6652
2
Su : 39827
Sd:  37969
Sf:  37971
Sd:  37970
Sf:  37987
Sd:  37971
Sf:  37972
Sd:  37972
Sf:  37973
Sd:  37977
Sf:  25384
Sd:  38062
Sf:  38042
Sd:  38096
Sf:  23770
Sd:  38132
Sf:  63383
Sd:  38290
Sf:  38291
Sd:  38291
Sf:  43773
Sd:  39656
Sf:  39657
Sd:  39659
Sf:  39660
Sd:  39660
Sf:  39661
Sd:  39714
Sf:  39715
Sd:  39715
Sf:  38117
Sd:  39806
Sf:  39807
Sd:  39827
Sf:  292
Sd:  39837
Sf:  39838
Sd:  39838
Sf:  39839
Sd:  

Sd:  51378
Sf:  51379
Sd:  51391
Sf:  51392
Sd:  51392
Sf:  24337
Sd:  51552
Sf:  1185
Sd:  51567
Sf:  51568
Sd:  51568
Sf:  51569
Sd:  51569
Sf:  51531
Sd:  52239
Sf:  65889
Sd:  52318
Sf:  52319
Sd:  52320
Sf:  52321
Sd:  52323
Sf:  52324
Sd:  52324
Sf:  52325
Sd:  52325
Sf:  52326
Sd:  52984
Sf:  52985
Sd:  52985
Sf:  52986
Sd:  52986
Sf:  52987
Sd:  52987
Sf:  52988
Sd:  53137
Sf:  53138
Sd:  53138
Sf:  38246
Sd:  53171
Sf:  47275
Sd:  55357
Sf:  53060
Sd:  55631
Sf:  55632
Sd:  55633
Sf:  55634
Sd:  57546
Sf:  19341
Sd:  57547
Sf:  31291
Sd:  57548
Sf:  57547
Sd:  57595
Sf:  57596
Sd:  57602
Sf:  34301
Sd:  57631
Sf:  57630
Sd:  57634
Sf:  57589
Sd:  57963
Sf:  38955
Sd:  58141
Sf:  58161


In [20]:
for c in C:
    print(c)

[23871, 23872]
[23872, 23861]
[46892, 23869]
[23869, 23870]
[23870, 23871]
[44946, 23862]
[152, 292, 6752, 834, 3533, 17, 2210, 6561, 828, 840, 6270, 832, 2875, 1776, 18, 2876, 2196, 830, 1777, 154, 826, 23867, 23868, 40132, 19180, 153, 16160, 47290, 6327, 17, 33487, 3604, 24349, 270, 23687]
[23868, 44946]
[153, 430, 559, 28327, 2231, 29745, 29746, 40029, 43197, 18933, 8609, 47693, 24709, 40359, 3565, 16159, 46888, 27282, 6658, 20706, 2872, 5166, 35041, 53124, 558, 27549, 1204, 83225, 19178, 36281, 1207, 24668, 31438, 5167, 6128, 37809, 7725, 47159, 41317, 173, 43406, 23718, 6264, 12635, 3373, 13421, 28827, 3157, 47124, 43421, 106, 35298, 45322, 28227, 27026, 3823, 1666, 9771, 31284, 5419, 3161, 32648, 30294, 45713, 5959, 5960, 6279, 6291, 7723, 6565, 4960, 410, 48415, 13333, 674, 14247, 23768, 12667, 2606, 32377, 55609, 12668, 3175, 21339, 6280, 2246, 29973, 1866, 5192, 30067, 8070, 6281, 3523, 22, 3940, 3829, 27833, 54, 14249, 6566, 2209, 6898, 39527, 43164, 8692, 675, 311, 21524, 40

[35065, 382]
[35110, 35111]
[35144, 24922]
[35265, 35266]
[35271, 35272]
[35272, 2529]
[35738, 35744]
[35769, 35688]
[35791, 51608]
[36096, 6564]
[36106, 36107]
[36152, 36153]
[36415, 36416]
[36416, 36417]
[36417, 36418]
[36435, 2875]
[36621, 10312]
[36651, 36652]
[36653, 9592]
[37428, 37415]
[37471, 37487]
[37483, 37484]
[37484, 5263]
[37624, 50001]
[37641, 37642]
[37642, 9064]
[37648, 37902]
[37650, 23795]
[37654, 41386]
[37673, 37674]
[37779, 37769]
[37782, 2254]
[37782, 6930]
[37801, 37802]
[37802, 37803]
[37804, 37805]
[37810, 2257]
[37827, 37828]
[37859, 24359]
[37973, 30012]
[37979, 46572]
[38109, 39713]
[38219, 38220]
[38247, 38228]
[38272, 38245]
[38276, 38272]
[38284, 43807]
[38293, 38224]
[39486, 39487]
[39516, 30422]
[39610, 2167]
[46855, 39619, 39610, 29107]
[39619, 39892]
[39744, 39745]
[39819, 39828]
[40067, 42885]
[40070, 40071]
[40071, 40072]
[40125, 46602]
[40126, 40142]
[40126, 43436]
[40332, 33285]
[40333, 40334]
[40344, 18432]
[41287, 41288]
[41300, 41301]
[41301, 

In [21]:
print(len(C))

1337


In [22]:
print(n)

[[  2   2   2 ...  -7   1  -8]
 [  1   1 -13 ...  -7  -6   1]
 [ -8   1   2 ...  -3   1  -3]
 ...
 [ -5  -5  -5 ...   1   1   1]
 [  1   1   1 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
